# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
!pip install torch==1.4.0
!pip3 install transformers==3.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.4 MB 6.2 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.4.0 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.4.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.4.0 which is incompatible.
fastai 2.7.10 requires torch<1.14,>=1.7, but you have torch 1.4.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████

In [ ]:
!python -V

Python 3.7.15


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

fatal: destination path 'ru-gpts' already exists and is not an empty directory.


In [ ]:
!mkdir models/

mkdir: cannot create directory ‘models/’: File exists


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download files

In [ ]:
!wget -O train.txt https://www.dropbox.com/s/oa3v9c7g9bp40xw/train.txt?dl=0
!wget -O valid.txt https://www.dropbox.com/s/mworl3ld6r3bg62/valid.txt?dl=0

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=res.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

Downloading: 100% 608/608 [00:00<00:00, 465kB/s]
Downloading: 100% 1.71M/1.71M [00:01<00:00, 1.09MB/s]
Downloading: 100% 1.27M/1.27M [00:01<00:00, 975kB/s]
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Downloading: 100% 551M/551M [00:13<00:00, 39.5MB/s]
Epoch:   0% 0/5 [00:00<?, ?it/s]
Iteration:   0% 0/53 [00:00<?, ?it/s]
Iteration:   2% 1/53 [00:00<00:49,  1.05it/s]
Iteration:   4% 2/53 [00:01<00:45,  1.11it/s]
Iteration:   6% 3/53 [00:02<00:44,

## Evaluate model
### Generate with generate_transformers.py script.

In [ ]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/essays \
    --k=5 \
    --p=0.95 \
    --length=500 \
    --repetition_penalty=5

Context >>> Вкусный
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
Вкусный салат «Новогодний» с курицей и сыром Рецепт: Куриную грудку отварить до готовности, остудить.Вынуть мясо из бульона (можно использовать мясорубкой).Добавить кубиками нарезанный лук полукольцами или соломой порезанное колечко помидора без кожицы в миску выложить мелко нашинкованную зелень петрушки по вкусу посолить добавить тертый сыр натерть майонезом смешать все это со сметаной добавит еще 1 столовую ложку томатной пасты перемешать соус готов
Салат оливье для Нового года 2016 от ТМ Красная Глинка готова! Можно подавать к праздничному столу вместе любимым соусом Айоли Сдобные булочки готовы? Подавать горячими ароматными блинчиками-сметанниками посыпав сверху кондитерской крошкой :) Желаем вам вкусного новогоднего настроения!!!
Context >>> Traceback (most recent call last):
  File "ru-gpts/generate_transformers.py", line 268, in <module>
    main()
  File "ru-gpts/generate_transfor

### Generate mannualy

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tok = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/models/essays")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/models/essays")

In [ ]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
text = "Название: "
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt, max_length=150, repetition_penalty=10.0, do_sample=True, top_k=2500, top_p=0.8, temperature=200.0)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tok.decode(out[0])

'Название:  ПШЕНА ИЗ МАРИНОВАННОЙ СЁМГИ Рецепт Чтобы приготовить пшено из маринованной сёмги, для начала необходимо сварить 500 миллилитров мясного бульона.Завернуть 400 грамм картофеля в кастрюлю и остудить до полуготовности 10-15 минут на слабом огне без крышки.Варить еще 5 -7 секунд после закипания готового супа ТТ «Эстадия».Выложить мясо грудкой вниз во взбитые сливки (из расчета 1 чайная ложка желатина).150 граммов мёда заварите при помощи сотейника или вымесьте руками, если будете использовать пену НЗ10%, чтобы прикрыть вареную массу'

In [ ]:
len(tok.decode(out[0]))

515

In [ ]:
!python /content/ru-gpts/convert2huggingface.py \
  --load /path/to/save/dir/ \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --max-position-embeddings 2048 \
  --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
  --no-load-optim \
  --export-huggingface /path/to/converted/checkpoint

Traceback (most recent call last):
  File "/content/ru-gpts/convert2huggingface.py", line 20, in <module>
    from generate_samples import setup_model, prepare_tokenizer
  File "/content/ru-gpts/generate_samples.py", line 24, in <module>
    from src import mpu
  File "/content/ru-gpts/src/mpu/__init__.py", line 35, in <module>
    from .layers import ColumnParallelLinear
  File "/content/ru-gpts/src/mpu/layers.py", line 28, in <module>
    from apex.normalization.fused_layer_norm import FusedLayerNorm as LayerNorm
ModuleNotFoundError: No module named 'apex'
